This model is flawed from the start!

It is taking in known data ('Open', 'Volume') from the current day, and attempting to solve for the current day's 'Close' price

It should actually be taking in the previous day's features ('Open', 'Volume') and attempting to solve for the current day's 'Close' price

In [252]:
# load all relevant imports and download CSV from yfinance API call
import pandas as pd
import requests 
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from datetime import date

# unsafe_session = requests.session()
# unsafe_session.verify = False

def load_data(tickerSymbols):
    yf.download(tickers=tickerSymbols
                # , session=unsafe_session
                ).to_csv(f'./csv/{tickerSymbols}_data.csv')

    # load data into DataFrame
    return pd.read_csv(f'./csv/{tickerSymbols}_data.csv')

def prediction_csv(dataframe, ticker):
    dataframe.to_csv(f'./predict-csv/{ticker}_data.csv')

def add_line_to_file(file_path, new_line):

    with open(file_path, "a") as file:
        file.write(new_line + "\n")


# Feature Engineering the Model

Here is where we separate the adolescents from the adults!

We need to predict TODAY's 'Open' and 'Close' price based on YESTERDAY's meaningful trading information


In [ ]:
# get only data rows and check columns for features
ticker = 'AMZN'
data_rows_only = load_data(ticker).iloc[2:]

df_shifted = data_rows_only.shift(1)

# rename the columns before concat()
df_shifted.columns = ['Price_prev', 'Close_prev', 'High_prev', 'Low_prev', 'Open_prev', 'Volume_prev']

df_combined = pd.concat([data_rows_only.loc[:,['Price','Close']], df_shifted], axis=1)
df_combined.columns = ['Current Date', 'Current Close','Day_prev', 'Close_prev', 'High_prev', 'Low_prev', 'Open_prev', 'Volume_prev']

print(len(df_shifted))


In [254]:
# build stock model and features
stock_model = RandomForestRegressor(random_state=1)
stock_features = ['Close_prev', 'High_prev', 'Low_prev', 'Open_prev', 'Volume_prev']


In [ ]:
# establish X (rows to analyze) and y (value to predict) variables
X = df_combined.iloc[1:][stock_features]
y = df_combined.iloc[1:]['Current Close']

print(len(X))

# Split data set into different variables:

# --- this is done to illustrate how well the model performs on FACTUAL outcomes.

# --- for example, we know the 'Open' value of our target (y) actually occurred 
# --- and can train our model on the actual features we feel are relevant of the day: (X)

# --- the goal is to train on sets train_X and train_y because they are characteristically SEPERATED
# --- this means the data hasn't trained on the sets on val_X and val_y
# --- it will then be making predictions based on data it has "never seen before"

# train_X = set of features used to train a machine learning model
# train_y = set of corresponding target values you want the model to use for predicting val_y later
# val_X = the features fed into the model to attempt to predict val_y
# val_y = 

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

# fit model on TRAINING data set, withhold validation set for comparison and training
# stock_model.fit(train_X,train_y)

# fit model on WHOLE data set
stock_model.fit(X,y)


Below is the meat of the transpositioning of the data.

Making predictions is one thing, but making sense of the values returned is another!!

In [256]:
# make "linear" predictions based on the features fed in line-by-line from val_X
val_predictions = stock_model.predict(val_X)

# turn this into a DataFrame for further processing / reporting
predict_df = pd.DataFrame(val_predictions)
predict_df.columns = ['Close Price Prediction'] # rename column for readability

# concatenate the predictions with the ACTUAL target values of val_y on ONE axis
concat_predict_and_val_y = pd.concat([predict_df, val_y.reset_index()], axis=1)

# Add new columns for classifying data
concat_predict_and_val_y.insert(1, 'Difference', 0) # the actual numerical difference from prediction to target value
concat_predict_and_val_y["Current Close"] = concat_predict_and_val_y["Current Close"].astype('float64') # cast type as 'float64' because it is an object
# prediction - actual gives us whether you LOST (-) or MADE (+) money that day
concat_predict_and_val_y["Difference"] = concat_predict_and_val_y["Close Price Prediction"] - concat_predict_and_val_y["Current Close"]

concat_predict_and_val_y.insert(1, 'Off-by Percentage', 0) # PERCENTAGE we were off from prediction to target value
# the difference in the $ value we lost/gained, divide this by the actual close price and we have the percentage lost/gained for that day
concat_predict_and_val_y["Off-by Percentage"] = concat_predict_and_val_y["Difference"] / concat_predict_and_val_y["Current Close"]

concat_predict_and_val_y = concat_predict_and_val_y.sort_values(by='Off-by Percentage', ascending=False)

prediction_csv(concat_predict_and_val_y, ticker)

# print(concat_predict_and_val_y)
# print(concat_predict_and_val_y.sum()["Difference"])


In [257]:
############################################
#         Derek Predictions Below          #
############################################

# print("Making a prediction for today based on yesterday's information:\n")

# print(df_combined[6972:6974])
# actual_todays_close = df_combined[6972:6974]['Current Close'].astype('float64')

# print("\nPrediction is:\n")

# todays_predict = stock_model.predict(df_combined[stock_features][6972:6974])[0].astype('float64')
# print(todays_predict)

# print("\nActual closed value:\n")
# print(actual_todays_close)

# diff = todays_predict - actual_todays_close
# print(diff)

# print("\ndifference:\n")
# print(diff / actual_todays_close)


In [ ]:
print("Lets make a prediction for 2/3/2025")

# gather the features from Friday 1/31/2025
search_date = pd.to_datetime('2025-01-31')

# within the original data, grab the row that corresponds
data_rows_only['Price'] = pd.to_datetime(data_rows_only['Price'])
search_date_row = data_rows_only[data_rows_only['Price'] == search_date]

# print(search_date_row.columns)

# change the columns to match stock_features variable
# stock_features = ['Close_prev', 'High_prev', 'Low_prev', 'Open_prev', 'Volume_prev']
search_date_row.columns = ['Price', 'Close_prev', 'High_prev', 'Low_prev', 'Open_prev', 'Volume_prev']


# print(search_date_row[stock_features])

prediction = stock_model.predict(search_date_row[stock_features])

prediction_txt = f"\n My model predicts that the value of '{ticker}' will be '{prediction[0]}' on '{search_date.date()}'"

print("Based on this row of values:")
print(search_date_row)
print(prediction_txt)


In [ ]:
print("That's a difference of:\n")
search_date_row.to_string()
prev_close = search_date_row["Close_prev"].astype('float64')
prediction_diff = (prediction - prev_close).iloc[0]
print(prediction_diff)


In [260]:
# add predictions to the file log!
formatted_date = date.today().strftime("%m/%d/%Y")
line_1 = formatted_date + " - \n\tBased on this row of values:\n"
line_4 = "That's a difference of:\n"

add_line_to_file("./predictions/predictions-log.txt", line_1)
add_line_to_file("./predictions/predictions-log.txt", search_date_row.to_string())
add_line_to_file("./predictions/predictions-log.txt", prediction_txt + "\n")
add_line_to_file("./predictions/predictions-log.txt", line_4)
add_line_to_file("./predictions/predictions-log.txt", f"${prediction_diff}\n")
